In [2]:
import os
from itertools import combinations
import dask
from dask import delayed
from dask.distributed import Client, progress
import dask.dataframe as dd
from glob import glob
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from scipy import sparse
import psutil
from pathlib import Path
NUM_WORKER = psutil.cpu_count(logical = False)
LIMIT = NUM_WORKER * 2
FP_processed  = 'processed/'
FP_matrices  = 'processed/matrices'
FP_b = 'interim/b_features/*'
FP_m = 'interim/m_features/*'
FP_A = 'processed/matrices/A'
Fp_B = 'processed/matrices/B'
Fp_P = 'processed/matrices/P'

In [3]:
from dask import delayed
from itertools import combinations
import psutil
NUM_WORKER = psutil.cpu_count(logical = False)
LIMIT = NUM_WORKER * 2

In [4]:
client = Client(n_workers = NUM_WORKER)

In [5]:
ROOT_DIR = '../'

In [6]:
from glob import glob

In [7]:
fp_b = '../data/tests/interim/b_features/*.csv'
fp_m = '../data/tests/interim/m_features/*.csv'

In [8]:
df_b = dd.read_csv(fp_b, usecols = ['api', 'block', 'app'], dtype = str)
df_m = dd.read_csv(fp_m, usecols = ['api', 'block', 'app'], dtype = str)
df = df_b.append(df_m).reset_index()

In [9]:
apis = df.api.unique().compute()
apis_dic = {apis[i]:i for i in range(len(apis))}

In [10]:
df['api_id'] = df.api.apply(lambda x: apis_dic[x], meta = int)

In [11]:
tmp = df.groupby('block').api_id\
    .apply(lambda x: list(combinations(x.drop_duplicates(), 2)), meta = list)\
    .explode()\
    .reset_index(drop = True)\
    .drop_duplicates()\
    .dropna().persist()

In [15]:
tmp.compute()

0            (1210, 690)
1            (1210, 691)
2          (1210, 10304)
3          (1210, 10305)
4         (1210, 259182)
               ...      
138918    (32968, 33003)
138948    (56833, 11671)
138952    (19675, 20737)
138954      (290, 20737)
138956    (20737, 19677)
Length: 5202007, dtype: object

In [13]:
progress(tmp)

VBox()

In [20]:
entries = pd.DataFrame(tmp.values.tolist()).values

In [22]:
values = np.full(shape=entries.shape[0], fill_value=1, dtype='i1')

In [32]:
result = sparse.coo_matrix(
                (values, (entries[:,0], entries[:,1])), shape=(len(apis), len(apis))
    )
result.setdiag(1)

In [33]:
result

<308822x308822 sparse matrix of type '<class 'numpy.int8'>'
	with 5510829 stored elements in COOrdinate format>